In [ ]:
print("hello")

hello


In [1]:
import os
os.chdir('../')

In [2]:
from src.utils.minio_uploader import MinioUploader
from src.utils.delta_exporter import *
from src.components.data_ingestion import *
import logging
import csv
from datetime import datetime, timedelta, timezone
from binance_sdk_c2c.c2c import C2C, ConfigurationRestAPI, C2C_REST_API_PROD_URL
from turtle import home
from typing import List, Optional
from binance_sdk_c2c.rest_api.models import GetC2CTradeHistoryResponse, GetC2CTradeHistoryResponseDataInner
from src.utils.utils import *
from src.components.data_ingestion import C2CExtended

# Test

In [3]:
from binance_common.configuration import ConfigurationRestAPI
# Create configuration for the REST API
configuration_rest_api = ConfigurationRestAPI(
    api_key=os.getenv("API_KEY", ""),
    api_secret=os.getenv("API_SECRET", ""),
    base_path=os.getenv("BASE_PATH", C2C_REST_API_PROD_URL),
)
c2c = C2CExtended(configuration_rest_api)
# Ví dụ 5: Lấy giao dịch khoảng thời gian tùy chỉnh (01/09/2025 - 10/09/2025)
# Lấy giao dịch cho danh sách các ngày bất kỳ
data = c2c.get_latest_by_week()

INFO:root:Page 1 rate limits: []
INFO:root:Page 1 retrieved 50 records, 50 within time range
INFO:root:Page 2 rate limits: []
INFO:root:Page 2 retrieved 50 records, 50 within time range
INFO:root:Page 3 rate limits: []
INFO:root:Page 3 retrieved 50 records, 50 within time range
INFO:root:Page 4 rate limits: []
INFO:root:Page 4 retrieved 50 records, 50 within time range
INFO:root:Page 5 rate limits: []
INFO:root:Page 5 retrieved 50 records, 50 within time range
INFO:root:Page 6 rate limits: []
INFO:root:Page 6 retrieved 50 records, 50 within time range
INFO:root:Page 7 rate limits: []
INFO:root:Page 7 retrieved 50 records, 50 within time range
INFO:root:Page 8 rate limits: []
INFO:root:Page 8 retrieved 50 records, 50 within time range
INFO:root:Page 9 rate limits: []
INFO:root:Page 9 retrieved 50 records, 50 within time range
INFO:root:Page 10 rate limits: []
INFO:root:Page 10 retrieved 50 records, 50 within time range
INFO:root:Page 11 rate limits: []
INFO:root:Page 11 retrieved 50 rec

## Convert API's response data to deltalake

In [4]:
len(data)

893

In [5]:
# 1) Convert API results (list[dict] or objects) to DataFrame
df = api_results_to_dataframe(data)


2025-10-18 17:24:25.939 | INFO     | src.utils.delta_exporter:api_results_to_dataframe:50 - Created DataFrame: shape=(893, 18)


In [6]:
df.shape

(893, 18)

In [8]:
delta_path = "test_batch_transformation/delta/c2c_trades"

In [ ]:
# Fix cho Windows: Set HADOOP_HOME để tránh lỗi
import os
import sys

# Tạo thư mục tạm cho Hadoop nếu đang dùng Windows
if sys.platform.startswith('win'):
    import tempfile
    hadoop_home = os.path.join(tempfile.gettempdir(), 'hadoop')
    os.makedirs(os.path.join(hadoop_home, 'bin'), exist_ok=True)
    os.environ['HADOOP_HOME'] = hadoop_home
    print(f"Set HADOOP_HOME to: {hadoop_home}")


In [ ]:
# Tạo SparkSession với cấu hình cho Windows
spark = SparkSession.builder \
    .appName("C2C Trade Analysis") \
    .master("local[*]") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0") \
    .config("spark.sql.warehouse.dir", "file:///D:/Working/binance-merchant-trading-flow/spark-warehouse") \
    .getOrCreate()

print(f"✓ Spark version: {spark.version}")
print(f"✓ Spark running successfully!")


In [ ]:
# Đọc file parquet từ Delta Lake
delta_path = "test_batch_transformation/delta/c2c_trades"

# Đọc Delta table
df_spark = spark.read.format("delta").load(delta_path)

print(f"Number of records: {df_spark.count()}")
print(f"Number of columns: {len(df_spark.columns)}")


In [ ]:
# Hiển thị schema của DataFrame
df_spark.printSchema()


In [ ]:
# Hiển thị 10 bản ghi đầu tiên
df_spark.show(10, truncate=False)


In [ ]:
# Hoặc nếu bạn chỉ muốn đọc trực tiếp file parquet (không qua Delta Lake)
# df_spark = spark.read.parquet("test_batch_transformation/delta/c2c_trades/data.parquet")
# df_spark.show(10)


In [ ]:
# Thống kê cơ bản
df_spark.describe().show()


In [ ]:
# Chuyển đổi sang Pandas DataFrame để xem dữ liệu dễ hơn (chỉ với dataset nhỏ)
df_pandas = df_spark.toPandas()
df_pandas.head(10)


In [9]:
write_dataframe_to_delta(df, delta_path, mode="append", partition_by=None)

2025-10-18 15:12:40.936 | INFO     | src.utils.delta_exporter:write_dataframe_to_delta:85 - Wrote DataFrame to Delta: path=test_batch_transformation/delta/c2c_trades, mode=append, rows=893


'test_batch_transformation/delta/c2c_trades'

In [10]:
test_bucket = "spark_bucket"

In [11]:
uploader = MinioUploader(endpoint = "localhost:9000", access_key = "adminc2c", secret_key = "adminc2c", secure = False, region = None)

In [18]:
upload_delta_to_minio(delta_path = delta_path, uploader = uploader, bucket = test_bucket, dest_prefix = "test_data_lakehouse/delta/c2c_trades")

2025-10-17 16:55:42.096 | INFO     | src.utils.minio_uploader:ensure_bucket:30 - Created bucket: test-bucket


2025-10-17 16:55:42.158 | INFO     | src.utils.minio_uploader:upload_file:57 - Uploaded file d:\Working\binance-merchant-trading-flow\test_data_lakehouse\delta\c2c_trades\part-00001-56781e34-cd46-48fe-ad42-0b397e9c14ca-c000.snappy.parquet -> s3://test-bucket/test_data_lakehouse/delta/c2c_trades/part-00001-56781e34-cd46-48fe-ad42-0b397e9c14ca-c000.snappy.parquet
2025-10-17 16:55:42.220 | INFO     | src.utils.minio_uploader:upload_file:57 - Uploaded file d:\Working\binance-merchant-trading-flow\test_data_lakehouse\delta\c2c_trades\part-00001-fef68d1f-e4de-4ccd-b223-3e84782218c0-c000.snappy.parquet -> s3://test-bucket/test_data_lakehouse/delta/c2c_trades/part-00001-fef68d1f-e4de-4ccd-b223-3e84782218c0-c000.snappy.parquet
2025-10-17 16:55:42.281 | INFO     | src.utils.minio_uploader:upload_file:57 - Uploaded file d:\Working\binance-merchant-trading-flow\test_data_lakehouse\delta\c2c_trades\_delta_log\00000000000000000000.json -> s3://test-bucket/test_data_lakehouse/delta/c2c_trades/_delta_

# Spark Transformation - Đọc và xử lý Parquet files

In [ ]:
# Import necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import os
import sys

In [ ]:
# Fix cho Windows: Set HADOOP_HOME để tránh lỗi
# Tạo thư mục tạm cho Hadoop nếu đang dùng Windows
if sys.platform.startswith('win'):
    import tempfile
    hadoop_home = os.path.join(tempfile.gettempdir(), 'hadoop')
    os.makedirs(os.path.join(hadoop_home, 'bin'), exist_ok=True)
    os.environ['HADOOP_HOME'] = hadoop_home
    print(f"✓ Set HADOOP_HOME to: {hadoop_home}")


In [ ]:
# Tạo SparkSession với cấu hình cho Windows
spark = SparkSession.builder \
    .appName("C2C Trade Analysis") \
    .master("local[*]") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0") \
    .config("spark.sql.warehouse.dir", "file:///D:/Working/binance-merchant-trading-flow/spark-warehouse") \
    .getOrCreate()

print(f"✓ Spark version: {spark.version}")
print(f"✓ Spark running successfully!")


Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1139)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1125)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:489)
	at org.apache.spark.SparkContext.addFile(SparkContext.scala:1790)
	at org.apache.spark.SparkContext.$anonfun$new$16(SparkContext.scala:528)
	at org.apache.spark.SparkContext.$anonfun$new$16$adapted(SparkContext.scala:528)
	at scala.collection.immutable.List.foreach(List.scala:431)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:528)
	at org.apache.spark.api.java.JavaSparkC

In [ ]:
# Đọc file parquet từ Delta Lake
delta_path = "test_batch_transformation/delta/c2c_trades"

# Đọc Delta table
df_spark = spark.read.format("delta").load(delta_path)

print(f"✓ Number of records: {df_spark.count()}")
print(f"✓ Number of columns: {len(df_spark.columns)}")


In [ ]:
# Hiển thị schema của DataFrame
df_spark.printSchema()


In [ ]:
# Hiển thị 10 bản ghi đầu tiên
df_spark.show(10, truncate=False)


In [ ]:
# Các cách đọc parquet khác:

# Cách 1: Đọc trực tiếp file parquet (không qua Delta Lake)
# df_parquet = spark.read.parquet("test_batch_transformation/delta/c2c_trades/data.parquet")
# df_parquet.show(5)

# Cách 2: Đọc tất cả file parquet trong thư mục
# df_all = spark.read.parquet("test_batch_transformation/delta/c2c_trades/*.parquet")
# df_all.show(5)

# Cách 3: Đọc với schema cụ thể
# from pyspark.sql.types import StructType, StructField, StringType, DoubleType
# custom_schema = StructType([...])
# df_custom = spark.read.schema(custom_schema).parquet("path/to/file.parquet")


In [ ]:
# Chuyển đổi sang Pandas DataFrame để xem dữ liệu dễ hơn (chỉ với dataset nhỏ)
df_pandas = df_spark.limit(10).toPandas()
df_pandas


## Một số phép biến đổi với Spark DataFrame


In [ ]:
# Thống kê cơ bản
df_spark.describe().show()


In [ ]:
# Chọn một số cột cụ thể
df_select = df_spark.select("orderNumber", "fiatType", "totalPrice", "price", "createTime")
df_select.show(5)


In [ ]:
# Lọc dữ liệu
# Ví dụ: Lọc các giao dịch có fiatType là 'VND'
df_filtered = df_spark.filter(col("fiatType") == "VND")
print(f"Total VND transactions: {df_filtered.count()}")
df_filtered.show(5)


In [ ]:
# Group by và aggregate
# Tính tổng giao dịch theo fiatType
df_grouped = df_spark.groupBy("fiatType") \
    .agg(
        count("*").alias("total_transactions"),
        sum("totalPrice").alias("total_amount"),
        avg("totalPrice").alias("avg_amount")
    )
df_grouped.show()
